<a href="https://colab.research.google.com/github/slowhandc1ap/First_parquet/blob/main/exam3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise  

ให้นักศึกษาแปลงไฟล์ csv  people-2000000.csv ให้อยู่ในรูป parquet format แล้วบันทึกในชื่อไฟล์ person.parquet และดำเนินการดังต่อไปนี้


In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import pyarrow.compute as pc

In [2]:
from google.colab import drive
drive.mount('/content/drive') #ระวังเครื่องหมาย ‘ กับ ' ต่างกันนะคะ

Mounted at /content/drive


In [3]:
import pandas as pd
path = '/content/drive/MyDrive/Colab Notebooks/DataSet/people-2000000.csv'
dfp = pd.read_csv(path) ## Dataset is now stored in a Pandas Dataframe
dfp

,Index,User Id,First Name,Last Name,Sex,Email,Phone,Date of birth,Job Title
0,1,4defE49671cF860,Sydney,Shannon,Male,tvang@example.net,574-440-1423x9799,2020-07-09,Technical brewer
1,2,F89B87bCf8f210b,Regina,Lin,Male,helen14@example.net,001-273-664-2268x90121,1909-06-20,"Teacher, adult education"
2,3,Cad6052BDd5DEaf,Pamela,Blake,Female,brent05@example.org,927-880-5785x85266,1964-08-19,Armed forces operational officer
3,4,e83E46f80f629CD,Dave,Hoffman,Female,munozcraig@example.org,001-147-429-8340x608,2009-02-19,Ship broker
4,5,60AAc4DcaBcE3b6,Ian,Campos,Female,brownevelyn@example.net,166-126-4390,1997-10-02,Media planner
...,...,...,...,...,...,...,...,...,...
1999995,1999996,AbBE4F3Eb5df843,Susan,Ellison,Male,yward@example.net,855.222.4135x62040,1913-01-05,Research scientist (maths)
1999996,1999997,e71af3E8De63CDF,Cole,Sherman,Female,nbarnett@example.org,(130)829-9159x800,1977-04-15,Marketing executive
1999997,1999998,fAF0618097acD1B,Ethan,Leonard,Male,brett57@example.com,514.898.2376x3069,1960-08-14,Photographer
1999998,1999999,f1FC3346F68DFAb,Jordan,Waller,Female,harveykendra@example.com,6920274656,1911-09-05,Chartered accountant


In [4]:
dfp.to_parquet('people.parquet')

In [16]:
table2= pa.Table.from_pandas(dfp)

In [44]:
table5 = pq.read_table('people.parquet',columns=['Index','First Name', 'Last Name','Sex','Date of birth','Job Title'])

In [45]:
pq.write_to_dataset(table5, root_path='Job_titleNew', partition_cols=['Job Title'])

In [46]:
table5

pyarrow.Table
Index: int64
First Name: string
Last Name: string
Sex: string
Date of birth: string
Job Title: string
----
Index: [[1,2,3,4,5,...,131068,131069,131070,131071,131072],[131073,131074,131075,131076,131077,...,262140,262141,262142,262143,262144],...,[1835009,1835010,1835011,1835012,1835013,...,1966076,1966077,1966078,1966079,1966080],[1966081,1966082,1966083,1966084,1966085,...,1999996,1999997,1999998,1999999,2000000]]
First Name: [["Sydney","Regina","Pamela","Dave","Ian",...,"Pam","Ana","Shelley","Darrell","Brian"],["Beth","Julian","Bradley","Robert","Katherine",...,"Brooke","Jeanette","Tami","Benjamin","Ronnie"],...,["Eugene","Margaret","Malik","Erik","Wesley",...,"Gabriela","Jill","Alan","Ivan","Shaun"],["Makayla","Jamie","Harold","Shannon","Isaiah",...,"Susan","Cole","Ethan","Jordan","Shelley"]]
Last Name: [["Shannon","Lin","Blake","Hoffman","Campos",...,"Holden","Branch","Garcia","Kelly","Guzman"],["Valencia","Hill","Pope","Hubbard","Sawyer",...,"Wilson","Serrano","Henso

In [48]:
# read partitioned table
table5 = pq.read_table('Job_titleNew')
table5.to_pandas()

,Index,First Name,Last Name,Sex,Date of birth,Job Title
0,131521,Kaitlyn,Baxter,Female,1943-09-24,Academic librarian
1,133596,Shelby,Henderson,Male,1919-06-15,Academic librarian
2,133800,Angel,Wu,Female,1991-02-20,Academic librarian
3,133935,Anthony,Haney,Female,2002-08-05,Academic librarian
4,134472,Henry,Hatfield,Female,1975-12-22,Academic librarian
...,...,...,...,...,...,...
1999995,1964579,Jaclyn,Lucas,Female,1946-05-10,Youth worker
1999996,1965049,Philip,Cooley,Female,1907-11-02,Youth worker
1999997,1965107,Kirsten,Frazier,Female,2014-12-20,Youth worker
1999998,1965658,April,Griffin,Male,1945-01-13,Youth worker


In [49]:
import pyarrow.compute as pc
grouped_table = table5.group_by("Job Title").aggregate(
  [("Index",
  "count",  #count #max #min

  pc.CountOptions(mode="only_valid"))]
)

grouped_table.to_pandas()

,Index_count,Job Title
0,3234,Academic librarian
1,3134,Accommodation manager
2,3185,"Accountant, chartered certified"
3,3130,"Accountant, chartered management"
4,3174,"Accountant, chartered public finance"
...,...,...
631,3092,Water quality scientist
632,3116,Web designer
633,2943,Wellsite geologist
634,3140,Writer


In [56]:
expr1 = pc.field("Sex") == "Male"

In [51]:
expr2 = pc.field("Job Title") == "Writer"

In [57]:
filtered_df = table5.filter(expr1 & expr2).to_pandas()  #descending


In [66]:
filtered_df

,Index,First Name,Last Name,Sex,Date of birth,Job Title
0,135552,Caitlyn,Lamb,Male,1976-08-18,Writer
1,137798,Curtis,Ellis,Male,1977-06-04,Writer
2,138066,Daniel,Spears,Male,1943-12-08,Writer
3,139488,Mark,Holt,Male,1943-05-24,Writer
4,142068,Jared,Ayers,Male,1909-04-07,Writer
...,...,...,...,...,...,...
1537,1962922,Albert,Miles,Male,1926-06-15,Writer
1538,1962975,Dale,Blankenship,Male,1911-04-01,Writer
1539,1963964,Logan,Owens,Male,2011-02-09,Writer
1540,1964304,Franklin,Daniels,Male,1909-01-10,Writer


In [67]:

table = table5.drop(["Index", "First Name","Date of birth"])
print(table)

pyarrow.Table
Last Name: string
Sex: string
Job Title: dictionary<values=string, indices=int32, ordered=0>
----
Last Name: [["Baxter","Henderson","Wu","Haney","Hatfield",...,"Riley","Mcdaniel","Wise","Oconnell","Potter"],["Mitchell","Jensen","Collier","Kline","Marsh",...,"Rich","Greer","Gilbert","Conner","Merritt"],...,["Butler","Mathews","Morales","Gilbert","Wolf",...,"Snow","Contreras","Larsen","Holland","Fitzpatrick"],["Hill","Sanford","Johnston","Mora","Hoover",...,"Lucas","Cooley","Frazier","Griffin","Manning"]]
Sex: [["Female","Male","Female","Female","Female",...,"Male","Male","Male","Female","Female"],["Male","Female","Male","Female","Male",...,"Female","Male","Male","Male","Female"],...,["Female","Male","Male","Female","Male",...,"Female","Male","Female","Male","Female"],["Female","Male","Male","Male","Female",...,"Female","Female","Female","Male","Female"]]
Job Title: [  -- dictionary:
["Academic librarian","Accommodation manager","Accountant, chartered certified","Accountant

In [68]:
table.to_pandas()

,Last Name,Sex,Job Title
0,Baxter,Female,Academic librarian
1,Henderson,Male,Academic librarian
2,Wu,Female,Academic librarian
3,Haney,Female,Academic librarian
4,Hatfield,Female,Academic librarian
...,...,...,...
1999995,Lucas,Female,Youth worker
1999996,Cooley,Female,Youth worker
1999997,Frazier,Female,Youth worker
1999998,Griffin,Male,Youth worker


In [69]:
expr1 = pc.field("Sex") == "Male"

In [70]:
expr2 = pc.field("Job Title") == "Writer"

In [71]:
final = table.filter(expr1 & expr2).to_pandas()  #descending


In [72]:
final

,Last Name,Sex,Job Title
0,Lamb,Male,Writer
1,Ellis,Male,Writer
2,Spears,Male,Writer
3,Holt,Male,Writer
4,Ayers,Male,Writer
...,...,...,...
1537,Miles,Male,Writer
1538,Blankenship,Male,Writer
1539,Owens,Male,Writer
1540,Daniels,Male,Writer
